In [18]:
import pyautogui
from PIL import Image, ImageDraw

import time
import pydirectinput
from directinput import PressKey, ReleaseKey

import math
import keyboard
import threading

In [4]:
def countdown(seconds, show_print="Blast Off!"):
    while seconds > 0:
        print(f"Countdown: {seconds} seconds remaining")
        time.sleep(1)  # Pause for 1 second
        seconds -= 1
    print(f"{show_print}")

In [5]:
def locate_client(client_img_path, is_testing=False):
    """
    client_img_path (String): Client Image Path
    is_testing (Boolean): For Testing, displays a screenshot where the client is found
    """
    try:
        # Locate the image on the screen
        client_location = pyautogui.locateOnScreen(client_img_path, confidence=0.8)

        # Region Adjustment Parameters
        client_adjust_left = 10
        client_adjust_top = 5
        client_adjust_width = 935
        client_adjust_height = 775

        # Apply Region adjustments
        x, y, width, height = (client_location.left - client_adjust_left,
                               client_location.top - client_adjust_top,
                               client_location.width + client_adjust_width,
                               client_location.height + client_adjust_height)

        # Testing mode visualization
        if is_testing:
            screenshot = pyautogui.screenshot()
            draw = ImageDraw.Draw(screenshot)
            draw.rectangle((x, y, x + width, y + height), outline='blue', width=3)
            screenshot.show()

        return {"Is_Found": True, "Region": (x, y, width, height)}

    except pyautogui.ImageNotFoundException:
        return {"Is_Found": False, "Region": (0, 0, 0, 0)}

In [6]:
def locate_mobs(mob_img_path, client_region, confidence_level=0.85,is_testing=False):

    try:

        matches = list(pyautogui.locateAllOnScreen(mob_img_path, confidence=confidence_level, region=client_region))

        center_coordinates = [(int(match.left + (match.width / 2)), int(match.top + (match.height / 2))) for match in matches]


        if is_testing:
            screenshot = pyautogui.screenshot()
            screenshot = screenshot.convert('RGB')
            draw = ImageDraw.Draw(screenshot)

            for match in matches:
                draw.rectangle((match.left, match.top, match.left + match.width, match.top + match.height), outline="black", width=2)

            print(f"{len(matches)}")
            screenshot.show()

        return {"Is_Found": True, "Coordinates": center_coordinates}

    except Exception as e:
        return {"Is_Found": False, "Coordinates": (0, 0, 0, 0)}

In [7]:
def remove_close_coordinates(coordinates, min_distance=3):
    cleaned_coordinates = []
    
    for coord in coordinates:
        x1, y1 = coord
        
        if all(math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2) >= min_distance for x2, y2 in cleaned_coordinates):
            cleaned_coordinates.append(coord)
    
    return cleaned_coordinates

In [8]:
def locate_player(player_img_path, is_testing=False):
    
    adjustment_left = 45
    adjustment_top = 120
    adjustment_width = 50

    try:
        location = pyautogui.locateOnScreen(player_img_path, confidence=0.8)
    
        # Initial Location
        center_x = location.left + location.width // 2
        center_y = location.top + location.height // 2
    
        # Adjusted Location
        red_box_top_left = (location.left - adjustment_left, location.top - adjustment_top)
        red_box_bottom_right = (location.left + location.width + adjustment_width, location.top + location.height)
        # New Location
        red_box_center_x = (red_box_top_left[0] + red_box_bottom_right[0]) // 2
        red_box_center_y = (red_box_top_left[1] + red_box_bottom_right[1]) // 2
    
        if is_testing:
            screenshot = pyautogui.screenshot()
            screenshot = screenshot.convert("RGB")
            draw = ImageDraw.Draw(screenshot)
            draw.rectangle([red_box_top_left, red_box_bottom_right], outline="red", width=3)
            screenshot.show()
    
        return {"Is_Found": True, "Coordinates": (red_box_center_x, red_box_center_y)}
        
    except pyautogui.ImageNotFoundException:
        return {"Is_Found": False, "Coordinates": (0, 0)}

In [9]:
def find_nearest_coordinate(player_coord, mob_coords):
    if not mob_coords:
        return None  # Return None if the list of mob coordinates is empty
    
    nearest_coord = None
    min_distance = float('inf')

    for mob_coord in mob_coords:
        # Calculate Euclidean distance
        distance = math.sqrt((mob_coord[0] - player_coord[0])**2 + (mob_coord[1] - player_coord[1])**2)
        if distance < min_distance:
            min_distance = distance
            nearest_coord = mob_coord

    return nearest_coord

In [10]:
def sort_coordinates_by_distance(player_coord, mob_coords):
    if not mob_coords:
        return []  # Return an empty list if the list of mob coordinates is empty
    
    # Sort the mob_coords list by distance using a lambda function
    sorted_coords = sorted(
        mob_coords,
        key=lambda mob_coord: math.sqrt((mob_coord[0] - player_coord[0])**2 + (mob_coord[1] - player_coord[1])**2)
    )
    
    return sorted_coords

In [11]:
def do_attack(mob_coordinates):

    if isinstance(mob_coordinates, list):
        for coordinates in mob_coordinates:
            PressKey(0x3B) #F1
            ReleaseKey(0x3B)
            pydirectinput.moveTo(*coordinates)
            pyautogui.mouseDown()
            time.sleep(0.001)
            pyautogui.mouseUp()
    else:
        PressKey(0x3B) #F1
        ReleaseKey(0x3B)
        pydirectinput.moveTo(*mob_coordinates)
        pyautogui.mouseDown()
        time.sleep(0.001)
        pyautogui.mouseUp()

In [12]:
def do_teleport():

    check_teleport = 'Teleporter.jpg'

    time.sleep(0.1)
    PressKey(0x42) # F8
    time.sleep(0.1)
    ReleaseKey(0x42)
    time.sleep(0.6)

    pyautogui.useImageNotFoundException()

    # Check if teleporter is on
    while True:
        try:
            is_teleporter_on = pyautogui.locateOnScreen(check_teleport, confidence=0.8)
            if is_teleporter_on:
                PressKey(0x1C) # Enter
                ReleaseKey(0x1C)
            else:
                break
        except Exception as e:
            print(f"Teleporter not found")
            break

    # Make sure chatbox not on
    do_removechatbox()

In [13]:
def do_removechatbox():

    check_chatbar = 'Chatbar.jpg'
    
    #time.sleep(0.5)
    # Check if chatbar is on
    pyautogui.useImageNotFoundException()
    while True:
        try:
            is_chatter_on = pyautogui.locateOnScreen(check_chatbar, confidence=0.8)
            if is_chatter_on:
                PressKey(0x1C) # Enter
                #ReleaseKey(0x1C)
            else:
                break
        except Exception as e:
            print(f"Chatbox not found")
            break

In [14]:
def do_heal(coords):
    pydirectinput.moveTo(*coords)
    PressKey(0x3E) #F4
    ReleaseKey(0x3E)
    pyautogui.mouseDown()
    time.sleep(0.001)
    pyautogui.mouseUp()

In [20]:
def do_loot(just_loot=True):

    if just_loot:

        PressKey(0x3D)
        time.sleep(0.01)
        ReleaseKey(0x3D)

    else:

        check_card = 'LootCard.png'
        check_etherdust = 'LootEtherDust.png'
    
        # Ensure the ImageNotFoundException is enabled
        pyautogui.useImageNotFoundException()
    
        try:
            # Locate items on the screen
            found_items = {
                "card": pyautogui.locateOnScreen(check_card, confidence=0.8),
                "etherdust": pyautogui.locateOnScreen(check_etherdust, confidence=0.8)
            }
    
            # Check if any of the items were found
            if any(found_items.values()):
                time.sleep(0.02)
                PressKey(0x3D)  # Press F3
                time.sleep(0.02)
                ReleaseKey(0x3D)  # Release F3
    
        except Exception as e:
            # Log any errors that occur
            print(f"Loot not found")

In [17]:
def do_buff():
    
    time.sleep(0.001)
    PressKey(0x02)
    time.sleep(0.001)
    ReleaseKey(0x02)

    time.sleep(0.001)
    PressKey(0x02)
    time.sleep(0.001)
    ReleaseKey(0x02)
    
    time.sleep(0.5)
    
    time.sleep(0.001)
    PressKey(0x03)
    time.sleep(0.001)
    ReleaseKey(0x03)

    time.sleep(0.5)
    
    time.sleep(0.001)
    PressKey(0x04)
    time.sleep(0.001)
    ReleaseKey(0x04)

    time.sleep(0.6)
    
    time.sleep(0.001)
    PressKey(0x05)
    time.sleep(0.001)
    ReleaseKey(0x05)

    time.sleep(0.5)
    
    time.sleep(0.001)
    PressKey(0x06)
    time.sleep(0.001)
    ReleaseKey(0x06)

    time.sleep(0.5)
    
    time.sleep(0.001)
    PressKey(0x3F)
    time.sleep(0.001)
    ReleaseKey(0x3F)

    time.sleep(0.5)
    
    time.sleep(0.001)
    PressKey(0x40)
    time.sleep(0.001)
    ReleaseKey(0x40)

    #time.sleep(1)
    #do_teleport()

# Version 1

In [46]:
countdown(2)

get_client_region = locate_client(client_img_path="FindClient.png", is_testing=False)
get_locate_player = locate_player("HPMPBar.jpg", is_testing=False)

while True:
        mob_regions = locate_mobs(mob_img_path="MobIndicator.png",
                                  confidence_level=0.8,
                                  client_region=get_client_region["Region"],
                                  is_testing=False)
        if not mob_regions["Is_Found"]:
            #print("No Mob Found. Teleporting...")
            do_heal(get_locate_player["Coordinates"])
            time.sleep(0.1)
            do_teleport()
            #time.sleep(0.1)
            do_removechatbox()
            
            continue  # Restart the loop
        
        # Remove close coordinates and find the nearest mob
        mob_regions["Coordinates"] = remove_close_coordinates(mob_regions["Coordinates"], min_distance=15)
        #get_target_mob = find_nearest_coordinate(get_locate_player["Coordinates"], mob_regions["Coordinates"])
        get_target_mob = find_nearest_coordinate(get_locate_player["Coordinates"], mob_regions["Coordinates"])
        # Execute attack
        attack_successful = do_attack(get_target_mob)
        do_loot()
        if attack_successful:
            do_loot()
            print(f"Attack executed on target at {get_target_mob}.")
        else:
            print("Attack failed. Breaking the loop.")
            #do_heal(get_locate_player["Coordinates"])
            #time.sleep(0.1)
            #do_teleport()
            #time.sleep(0.1)
            #do_removechatbox()
            continue

Countdown: 2 seconds remaining
Countdown: 1 seconds remaining
Blast Off!
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Teleporter not found
Chatbox not found
Chatbox not found
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Teleporter not found
Chatbox not found
Chatbox not found
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Teleporter not found
Chatbox not found
Chatbox not found
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Attack f


KeyboardInterrupt



# Version 2

What I'm currently Running

In [25]:

# Define global variables
heal = 0
loot_check = 0
buff = 0

def reset_counter_for_heal():
    global counter  # Access the global counter variable
    while True:
        heal = 1  # Reset the counter to 1
        print("Heal reset to 1.")
        time.sleep(30)  # Wait for 30 seconds

def reset_counter_every_for_loot():
    global loot_check  # Access the global loot_check variable
    while True:
        loot_check = 1  # Reset loot_check to 1
        print("Loot_check reset to 1.")
        time.sleep(20)  # Wait for 20 seconds

def reset_counter_every_for_buff():
    global buff  # Access the global loot_check variable
    while True:
        buff = 1  # Reset loot_check to 1
        print("Loot_check reset to 1.")
        time.sleep(60)  # Wait for 20 seconds

heal = 0
loot_check = 0
buff = 0

# Create and start separate threads
reset_counter_thread = threading.Thread(target=reset_counter_for_heal, daemon=True)
reset_counter_thread.start()

reset_loot_thread = threading.Thread(target=reset_counter_every_for_loot, daemon=True)
reset_loot_thread.start()

reset_loot_thread = threading.Thread(target=reset_counter_every_for_buff, daemon=True)
reset_loot_thread.start()

countdown(2)

get_client_region = locate_client(client_img_path="FindClient.png", is_testing=False)
get_locate_player = locate_player("HPMPBar.jpg", is_testing=False)

while True:
    # Check if 'Esc' key is pressed
    if keyboard.is_pressed("esc"):
        print("Script stopped by user.")
        break

    if keyboard.is_pressed("`"):
        do_heal(get_locate_player["Coordinates"])
        continue

    if keyboard.is_pressed("="):
        do_buff()
        continue

    mob_regions = locate_mobs(mob_img_path="MobIndicator.png",
                              confidence_level=0.6,
                              client_region=get_client_region["Region"],
                              is_testing=False)
    if not mob_regions["Is_Found"]:

        if heal == 1:
            do_heal(get_locate_player["Coordinates"])
            do_buff()
            heal = 0
        if loot_check == 1:
            do_loot(just_loot=True)
            loot_check = 0

        if buff == 1:
            do_loot(just_loot=True)
            buff = 0

        continue  # Restart the loop

    mob_regions["Coordinates"] = remove_close_coordinates(mob_regions["Coordinates"], min_distance=5)
    get_target_mob = sort_coordinates_by_distance(get_locate_player["Coordinates"], mob_regions["Coordinates"])[:1]

    attack_successful = do_attack(get_target_mob)

    counter = counter + 1

    if attack_successful:
        print(f"Attack executed on target at {get_target_mob}.")
    else:
        loot_once = 1
        print("Attack failed. Breaking the loop.")
        continue

Heal reset to 1.
Loot_check reset to 1.
Loot_check reset to 1.
Countdown: 2 seconds remaining
Counter reset to 1.
Countdown: 1 seconds remaining
Heal reset to 1.
Blast Off!
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Loot_check reset to 1.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Heal reset to 1.
Counter reset to 1.
Attack failed. Breaking the loop.
Loot_check reset to 1.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Loot_check reset to 1.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Attack failed. Breaking the loop.
Loot_check reset to 1.
Attack failed. Breaking the loop.
Attack failed. Bre

In [42]:
a = [1,2]

In [43]:
a[:3]

[1, 2]

# Prototype

In [96]:
countdown(2)

get_client_region = locate_client(client_img_path="FindClient.png",
                                  is_testing=False
                                 )

get_locate_player = locate_player("HPMPBar.jpg", is_testing=False)

if get_client_region["Is_Found"]:
    
    mob_regions = locate_mobs(mob_img_path="MobIndicator.png",
                              confidence_level=0.8,
                              client_region=get_client_region["Region"],
                              is_testing=False
                             )
    if mob_regions["Is_Found"]:
        mob_regions["Coordinates"] = remove_close_coordinates(mob_regions["Coordinates"], min_distance=15)
        get_target_mob = find_nearest_coordinate(get_locate_player["Coordinates"], mob_regions["Coordinates"])
        #pydirectinput.moveTo(*get_target_mob)
        do_attack(get_target_mob)
    else:
        print("No Mob Found")
        pass
        do_teleport()
else:
    print("NotFound")

Countdown: 2 seconds remaining
Countdown: 1 seconds remaining
Blast Off!


In [56]:
mob_regions["Coordinates"]

[(1459, 605), (1034, 685), (1422, 1006)]

In [17]:
mob_regions["Coordinates"]

[(1420, 494), (1167, 732), (1182, 732), (1280, 1080)]

# Sample Main Py

In [ ]:
def main_loop():
    while True:
        # Locate the client region
        get_client_region = locate_client(client_img_path="FindClient.png", is_testing=False)
        if not get_client_region["Is_Found"]:
            print("Client not found. Exiting loop.")
            break
        
        # Locate the player's position
        get_locate_player = locate_player("HPMPBar.jpg", is_testing=False)
        if not get_locate_player["Is_Found"]:
            print("Player not found. Exiting loop.")
            break
        
        # Locate mobs within the client region
        mob_regions = locate_mobs(mob_img_path="MobIndicator.png",
                                  confidence_level=0.8,
                                  client_region=get_client_region["Region"],
                                  is_testing=False)
        if not mob_regions["Is_Found"]:
            print("No Mob Found. Teleporting...")
            do_teleport()
            continue  # Restart the loop
        
        # Remove close coordinates and find the nearest mob
        mob_regions["Coordinates"] = remove_close_coordinates(mob_regions["Coordinates"], min_distance=15)
        get_target_mob = find_nearest_coordinate(get_locate_player["Coordinates"], mob_regions["Coordinates"])
        
        # Execute attack
        attack_successful = do_attack(get_target_mob)
        if attack_successful:
            print(f"Attack executed on target at {get_target_mob}.")
        else:
            print("Attack failed. Breaking the loop.")
            break
        
        # Optional delay to prevent excessive CPU usage
        time.sleep(0.5)  # Adjust the delay as needed

# Run the main loop
if __name__ == "__main__":
    main_loop()